In [1]:
import numpy as np
import math 
from itertools import combinations
import gmpy2


def factors(n): #розклад числа на прості множники
    primfact = []
    if n < 0 : 
        primfact.append(-1)
        n = -1 * n
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfact.append(d)  
            n //= d
        d += 1
    if n > 1:
        primfact.append(n)
    return primfact

def update_vec(b_i, Base): #перетворюємо b на вектор v
    fact = factors(b_i)
    v = [0 for i in range(len(Base))]
    if (len(fact) == 1) or (len(fact) == 2 and fact[0] == -1):#якщо b просте або -просте
        return v
    for i in range(len(Base)): 
        if Base[i] in fact:
            v[i] = fact.count(Base[i])
    return v


def num_to_vec(b_lst, Base): #перевіряємо чи є и B-числом і розклад. по базі
    V = []
    for i in range(len(b_lst)):
        fact = factors(b_lst[i]) #розклад b^2 на прості множники
        if (len(fact) == 1) or (len(fact) == 2 and fact[0] == -1):#якщо b просте або -просте
            v_i = [0 for i in range(len(Base))]
            V.append(v_i)
        elif not all(el in Base for el in fact): #перевіряємо чи є ці множники у факторній базі
            v_i = [0 for i in range(len(Base))]
            V.append(v_i)
        else:    
            v_i = update_vec(b_lst[i], Base)
            V.append(v_i)
    V = np.array([np.array(vi) for vi in V])
    return V, Base


def display(lst): #красиво виводить список
    for i in lst:
        print(i)
        
def find_v(Vec): #перебір векторів, щоб знайти такі що в сумі = 0 (mod2)
    n = len(Vec)
    x = range(n)
    res = []
    for i in range(1, n+1):
        comb = list(combinations(x, i))
        for c in comb:
            no_zero = True
            s = [0 for i in range(len(Vec[0]))]
            for j in c:
                if not np.any(Vec[j]): #перевірка щоб не враховувати v=[00..0]
                    no_zero = False
                else: 
                    s = (s + Vec[j]) % 2
            if no_zero and np.sum(s) == 0:
                res.append(list(c))
    return res  

In [2]:
def find_roots(m, n, p, L): #solve q(x)=(x+m)^2-n=0 (mod p)
    q = lambda x: (x**2 + 2*m*x + m**2 - n)%p
    roots = []
    for x in range(-L, L+1):
        if q(x) == 0: roots.append(x)
    return roots

def what_to_sub(m, n, L, X, Base): #рахуємо що будемо віднімати
    res = [0 for i in range(2*L+1)]
    for p in Base[1:]:
        roots = find_roots(m, n, p, L)
#         print("p=",p)
#         print("roots=",roots)
        for r in roots:
            ind = X.index(r)
            res[ind] = res[ind] + round(math.log10(p),2)
    return res

In [3]:
def gen_primes(N): #генерує прості числа в інтервалі від 2 до N
    primes = []
    for num in range(2,N + 1):  
        isprime = True
        if num > 1:  
            for i in range(2,num):  
                if (num % i) == 0:  
                    isprime = False
                    break  
        if isprime == True:
            primes.append(num)
    return primes

    

def jacobi(n,m, t): #oбчислює символ Якобі (N/m), де N -- число яке факторизуємо.  t - знак
    if n == 1:
            return t
    if m%2 == 0: #перевірка що знаменник непарний
        return 0
    k = n % m
#     print(f"{t}({k}/{m})")
    if k == 1:
        return t
    primfact = factors(k) #розкладаємо k на множники
    i = list(primfact).count(2) #визначаємо який степінь двійки є дільником k
    if i%2 == 0: #якщо парний
        k = int(k / 2**i )
        if k == 1:
            return t
        else: 
            pow = (m-1)*(k-1)/4
            t_1 = int((-1)** pow)
#             print(f"{t_1 * t}({k}/{m})")
            return jacobi(m, k, t_1 * t)
    else: #якщо непарний, то розбиваємо на 2 символи якобі
        k = int(k / (2**i))
        pow_1 = (m**2 - 1) / 8
        pow_2 = (m-1)*(k-1)/4
        t_1 = int((-1)** pow_1)
        t_2 = int((-1)** pow_2)
#         print(f"{t_1}({k}/{m})")
        res = jacobi(m, k, t_1) * t_2 * t
        return res

In [4]:
def gen_base(n, B):
    Base = [-1, 2]
    primes = gen_primes(B) #генеруємо прості числа від 2 до N
    for el in primes[1:]:
        if gmpy2.legendre(n, el) == 1:
            Base.append(el)
    return Base

In [5]:
##з просіюванням
def quadratic_sieve(n, L):
    B = int(math.exp((1/2)*((math.log2(n))*(math.log2(math.log2(n))))**(1/2))) #границя до якої генерувати прості числа в базу
    print("B=",B)
    Base = gen_base(n, B)
    print("Base=", Base)
    m = int(math.sqrt(n))
    print("m=",m)
    counter = 0
    solved = False
    while not solved:
        print("------------------------")
        print("iteration=", counter)
        print("L=", L)
        X = [i for i in range(-L, L+1)]
        a = [(x_i+m) for x_i in X]
        b = [a_i**2-n for a_i in a]
        lg_b = [round(math.log10(abs(b_i)),2) for b_i in b] #округлення до 2 знаків після коми
        print("lg_b=",lg_b)
        logs = what_to_sub(m, n, L, X, Base)
        print("what_to_sub =",logs)
        sub = [round(lg_b[i] - logs[i], 2) for i in range(2*L+1)]
        print("subtracted=",sub)
        k = np.partition(sub, 1)[1] #беремо як початкову границю друге за величиною число 
        for i in np.arange(k, max(sub)+0.1, 0.1):
            bound = round(i,2) 
            arr = [i for i in range(len(sub)) if sub[i] <= bound] #індекси ел-тів, які потрапили в bound
            b_chosen = [0 for i in range(2*L+1)]
            for ar in arr: #обираємо b, які потрапили в bound
                b_chosen[ar] = b[ar]  
            Vect, Base = num_to_vec(b_chosen, Base) #переводимо їх у векторне представлення

            nums = find_v(Vect)  #знаходимо які вектори в сумі дають 0
            if len(nums) > 0:
                for el in nums:
                    X = 1
                    for j in el:
                        X = (X * a[j]) % n
                    k = 1
                    for j in el:
                        k = (k * b[j]) 
                    Y = int(math.sqrt(k))

                    p = math.gcd((X + Y) % n, n)
                    q = math.gcd((X - Y) % n, n)

                    if (p > 1 and p < n) and (q > 1 and q < n):
                        solved = True

                        print("------SOLVED--------")
                        print("a=",a)
                        print("b=",b)
                        print("L=",L)
                        print("arr=",arr)
                        print("bound=",bound)
                        print("------------------------")
                        for j in el:
                            print(f"v_{j}=",Vect[j])
                        print("A:", [a[j] for j in el])
                        print("B:", [b[j] for j in el])
                        print("X=", X)
                        print("Y=", Y)
                        return p, q, el, X, Y
                        break
        L += 1
        counter += 1


n = 9073
L = 5 #інтервал просіювання [-L; L]
pom_scr = quadratic_sieve(n, L)
print("ANSWER:")
print(f"{n}=", f"{pom_scr[0]}*{pom_scr[1]}")

B= 32
Base= [-1, 2, 3, 7, 11, 13, 29]
m= 95
------------------------
iteration= 0
L= 5
lg_b= [2.99, 2.9, 2.78, 2.63, 2.37, 1.68, 2.16, 2.53, 2.73, 2.86, 2.97]
what_to_sub = [0.85, 1.82, 2.79, 0.3, 0.48, 0.78, 2.1500000000000004, 1.63, 0.48, 2.26, 0.48]
subtracted= [2.14, 1.08, -0.01, 2.33, 1.89, 0.9, 0.01, 0.9, 2.25, 0.6, 2.49]
------SOLVED--------
a= [90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
b= [-973, -792, -609, -424, -237, -48, 143, 336, 531, 728, 927]
L= 5
arr= [1, 2, 5, 6, 7, 9]
bound= 1.11
------------------------
v_1= [1 3 2 0 1 0 0]
v_5= [1 4 1 0 0 0 0]
v_6= [0 0 0 0 1 1 0]
v_7= [0 4 1 1 0 0 0]
v_9= [0 3 0 1 0 1 0]
A: [91, 95, 96, 97, 99]
B: [-792, -48, 143, 336, 728]
X= 7633
Y= 1153152
ANSWER:
9073= 43*211


In [6]:
#без просіювання
def Pomeranz(n,Base):
    m = int(math.sqrt(n))
    L = 5
    i = 0
    solved = False
    while not solved:
        print("i=", i)
        print("L=", L)
        X = [i for i in range(-L, L+1)]
        a = [(x_i+m) for x_i in X]
        b = [a_i**2-n for a_i in a]
        Vect, Base = num_to_vec(b, Base)
        nums = find_v(Vect) 
        if len(nums) > 0:
            for el in nums:
                X = 1
                for j in el:
                    X = (X * a[j]) % n
                k = 1
                for j in el:
                    k = (k * b[j]) 
                Y = int(math.sqrt(k))
                
                p = math.gcd((X + Y) % n, n)
                q = math.gcd((X - Y) % n, n)

                if (p > 1 and p < n) and (q > 1 and q < n):
                    solved = True

                    display(Vect)
                    for j in el:
                        print(f"v_{j}=",Vect[j])
                    print("A:")
                    for j in el:
                        print(a[j])
                    print("B:")
                    for j in el:
                        print(b[j])
                    print("a=",a)
                    print("b=",b)
                    print("L=",L)
                    return p, q, el, X, Y
        L += 1
        i += 1

    
B = [-1, 2, 5, 7]
pom = Pomeranz(1649,B)
print("pom=", pom)

i= 0
L= 5
[0 0 0 0]
[0 0 0 0]
[1 3 1 1]
[0 0 0 0]
[1 7 0 0]
[1 0 0 2]
[0 5 0 0]
[0 0 0 0]
[0 3 2 0]
[0 0 0 0]
[0 0 0 0]
v_6= [0 5 0 0]
v_8= [0 3 2 0]
A:
41
43
B:
32
200
a= [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
b= [-424, -353, -280, -205, -128, -49, 32, 115, 200, 287, 376]
L= 5
pom= (97, 17, [6, 8], 114, 80)
